In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
import wbdata as wb
from contextlib import contextmanager
import pytz
from datetime import timedelta

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api
from workers.common import read_json, hash_table
import tradeDB

In [2]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.jsonc"},
            "STOCK": {"file": "./assets/stock.jsonc"},
            "ETF": {"file": "./assets/etf.jsonc"},
            "COMODITIES": {"file": "./assets/comodities.jsonc"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json( # type: ignore
        SECTORS[k]["file"]
    )

In [3]:
trade= tradeDB.Trader()
db_file = trade.db

In [4]:
reload(sql)

<module 'workers.sql' from '/home/mi/Dropbox/prog/python/tradeDB/workers/sql.py'>

In [18]:
dat = pd.read_csv('./dev/dat.csv')
dat = dat.apply(lambda x: pd.to_datetime(x) if re.search('date',x.name) else x)
tab='STOCK'
tabL=[tab+'_DESC',tab]

In [12]:
# merge with what we already know
known = sql.query(
        db_file=db_file,
        tab=tab,
        symbol=dat.loc[:, "symbol"].tolist(),
        from_date=date(1900,1,1),
        to_date=date.today()
    )
# add hash column
known['hash']=hash_table(known,tab)
known = known.apply(lambda x: pd.to_datetime(x) if re.search('date',x.name) else x)


In [13]:
known.loc[0,'info']='costam'
known

,industry,country,vol,date,from_date,info,symbol,val,to_date,name,hash
0,None,PL,None,2023-10-06,2023-10-06,costam,ACP,74.950,2023-10-06,ASSECOPOL,bf66636c7010aa2b3611165c364fac17
1,None,PL,None,2023-10-06,2023-10-06,None,ALE,28.750,2023-10-06,ALLEGRO,81e72552fe1dca121c25c8e83cc53c49
2,None,PL,None,2023-10-06,2023-10-06,None,ALR,55.740,2023-10-06,ALIOR,828491131bf307a86f8cda051f6ac09b
3,None,PL,None,2023-10-06,2023-10-06,None,CDR,107.600,2023-10-06,CDPROJEKT,b263eda51983feabe090a9a11f13fe4d
4,None,PL,None,2023-10-06,2023-10-06,None,CPS,11.430,2023-10-06,CYFRPLSAT,0fedeccaf3d740fedec223ea72c1aa20
5,None,PL,None,2023-10-06,2023-10-06,None,DNP,350.800,2023-10-06,DINOPL,b2af1a6920faa8b01d129712b146c1ed
6,None,PL,None,2023-10-06,2023-10-06,None,JSW,40.580,2023-10-06,JSW,d39f06f94b143236a70481655eed5e52
7,None,PL,None,2023-10-06,2023-10-06,None,KGH,107.400,2023-10-06,KGHM,ba7c40134b5d54b8d0ad86aeb9ae8043
8,None,PL,None,2023-10-06,2023-10-06,None,KRU,406.600,2023-10-06,KRUK,24b0c5cea875ec019eda3409c13881e5
9,None,PL,None,2023-10-06,2023-10-06,None,KTY,630.000,2023-10-06,KETY,0647b7c8a8d0c3ae8f6de8ea65ee5d8f


In [40]:
comp = dat.reindex(columns=known.columns) # align columns
comp = comp.merge(known, how='left', on=['hash'],suffixes=('','_known'))
# fill new data with what already known
for c in known.columns:
    if not re.search('(date)|(val)|(symbol)|(name)|(hash)',c):
        comp[c]=comp[c].fillna(comp[c+'_known'])
comp=comp.merge(known, how='outer', indicator=True)
new = comp.loc[comp['_merge']=='left_only']
rm = comp.loc[comp['_merge']=='right_only']

In [43]:
new.dropna()

,industry,country,vol,date,from_date,info,symbol,val,to_date,name,...,country_known,vol_known,date_known,from_date_known,info_known,symbol_known,val_known,to_date_known,name_known,_merge


In [42]:
rm

,industry,country,vol,date,from_date,info,symbol,val,to_date,name,...,country_known,vol_known,date_known,from_date_known,info_known,symbol_known,val_known,to_date_known,name_known,_merge
21,None,PL,None,2023-10-06,2023-10-06,costam,ACP,74.95,2023-10-06,ASSECOPOL,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaN,right_only
22,None,PL,None,2023-10-06,2023-10-06,None,ALE,28.75,2023-10-06,ALLEGRO,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaN,right_only
